In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
    break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Some libs and imports

In [2]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install pycocotools

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-6k1fyvl3
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-6k1fyvl3
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... - \ done
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl size=101740 sha256=9d7a43437244776c61f632f6f442bb882c2a19e86f4b31de878a1b3b2f1a488b
  Stored in directory: /root/.cache/pip/wheels/58/e6/f9/f87c8f8be098b51b616871315318329cae12cdb618f4caac93
Successfully built pycocotools


In [3]:
import os
import random
import json

from tqdm import tqdm
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import albumentations as A
import sys

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from collections import defaultdict
from segment_anything.utils.transforms import ResizeLongestSide

%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Image metadata

In [4]:
N_IMAGES = 512
TRAIN_IMAGE_SIZE = 512
INPUT_IMAGE_SIZE = (1920, 1080)

In [5]:
with open("../input/player-segmentation/instances_default.json") as f:
    annotations = json.load(f)

map_id_filename = {}
for index in range(len(annotations["images"])):
    map_id_filename[annotations["images"][index]["id"]] = annotations["images"][index]["file_name"]

### create SAM checkpoints

In [6]:
sam_checkpoint = "/kaggle/input/segment-anything/pytorch/vit-l/1/model.pth"
model_type = "vit_l"

device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.9,
    stability_score_thresh=0.9,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=120,  # Requires open-cv to run post-processing
)

predictor = SamPredictor(sam)

### Prepare data for training

In [7]:
# import pycocotools.mask as mask_util
# from matplotlib import pyplot as plt

# sam_annotations = dict()
# for img_id in tqdm(map_id_filename.keys()):
#     sam_annotations[img_id] = dict()
#     sam_annotations[img_id]['id'] = str(img_id)
#     sam_annotations[img_id]['annotations'] = list()
    
#     single_mask = None
#     for ann in annotations["annotations"]:
#         if ann['image_id'] == img_id:
#             item = dict()
#             item['type'] = 'player'
#             item['coordinates'] = list()

#             segmentation = ann["segmentation"]
# #             polygons = np.array(segmentation)
# #             polygon = polygon.reshape((-1, 1, 2)).astype(int)
#             rles = mask_util.frPyObjects(segmentation, 1080, 1920)
#             rle = mask_util.merge(rles)
#             gen_mask = mask_util.decode(rle)
#             #gen_mask = cv2.resize(gen_mask, dsize=(512, 512), interpolation=cv2.INTER_NEAREST)
            
#             if single_mask is None:
#                 single_mask = gen_mask
#             else:
#                 single_mask += gen_mask

#     single_mask = single_mask>0       
#     item['coordinates'].append(single_mask)
#     sam_annotations[img_id]['annotations'].append(item)

### Debug - make sure data is correct

In [8]:
# import pycocotools.mask as mask_util
# from matplotlib import pyplot as plt

# def display_image_along_with_segmentation_masks(img_id=111):
#     file_name = None
#     for im in annotations["images"]:
#         if im['id'] == img_id:
#             file_name = im['file_name']

#     image = cv2.cvtColor(
#         cv2.imread(f'../input/player-segmentation/0_500/images/{file_name}'), 
#         cv2.COLOR_BGR2RGB
#     )
#     plt.imshow(image)
#     plt.show()

#     single_mask = None
#     for ann in annotations["annotations"]:
#         if ann['image_id'] == img_id:
#             polygons = ann['segmentation']
#             rles = mask_util.frPyObjects(polygons, 1080, 1920)
#             rle = mask_util.merge(rles)
#             mask = mask_util.decode(rle)
#             if single_mask is None:
#                 single_mask = mask
#             else:
#                 single_mask += mask
    
#     single_mask = single_mask>0
#     plt.imshow(single_mask)
#     plt.show()
#     return single_mask
            
# single_mask = display_image_along_with_segmentation_masks(img_id = 14)

### Load dataset for traininig

In [9]:
# def get_bounding_box(ground_truth_map):
#     # get bounding box from mask
#     try:
#         y_indices, x_indices = np.where(ground_truth_map > 0)
#         x_min, x_max = np.min(x_indices), np.max(x_indices)
#         y_min, y_max = np.min(y_indices), np.max(y_indices)
#     except:
#         # some debug info
#         print('y_indices', y_indices)
#         print('x_indices', x_indices)
#         print('ground_truth_map', ground_truth_map)
#     # add perturbation to bounding box coordinates
#     H, W = ground_truth_map.shape
#     x_min = max(0, x_min - np.random.randint(0, 1))
#     x_max = min(W, x_max + np.random.randint(0, 1))
#     y_min = max(0, y_min - np.random.randint(0, 1))
#     y_max = min(H, y_max + np.random.randint(0, 1))
#     bbox = np.array([x_min, y_min, x_max, y_max])

#     return bbox

# def get_image_mask_and_bbox(img_ids):
#     # For every bounding box(prompt), find the original image and ground truth mask
#     bbox_coords = defaultdict(list)
#     ground_truth_masks = defaultdict(list)
#     transformed_data = defaultdict(dict)
#     for img_id in img_ids: 
#         image = cv2.imread(f"../input/player-segmentation/0_500/images/{map_id_filename[img_id]}")
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  

#         transform = ResizeLongestSide(sam.image_encoder.img_size)
#         input_image = transform.apply_image(image)
#         input_image_torch = torch.as_tensor(input_image, device=device)
#         transformed_image = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]  

#         input_image = sam.preprocess(transformed_image)
#         original_image_size = image.shape[:2]
#         input_size = tuple(transformed_image.shape[-2:])

#         transformed_data[img_id]['image'] = input_image[0]
#         transformed_data[img_id]['input_size'] = input_size
#         transformed_data[img_id]['original_image_size'] = original_image_size
        
#         for annotation in sam_annotations[img_id]["annotations"]:
#             ground_truth_masks[img_id].append(annotation['coordinates'][0])
#             bbox_coords[img_id].append(get_bounding_box(annotation['coordinates'][0]))

#     return transformed_data, ground_truth_masks, bbox_coords

In [10]:
# device = "cuda"
# from collections import defaultdict
# from segment_anything.utils.transforms import ResizeLongestSide

# class PSDataset(torch.utils.data.Dataset):
#     def __init__(self, img_ids, device):
#         self.img_ids = img_ids
#         self.device = device
#         results = get_image_mask_and_bbox(img_ids)
#         self.transformed_data, self.ground_truth_masks, self.bbox_coords = results 
        
#     def __len__(self):
#         return len(self.img_ids)

#     def __getitem__(self, idx):
#         img_id = self.img_ids[idx]
#         return self.transformed_data[img_id], self.ground_truth_masks[img_id], self.bbox_coords[img_id]

# train_len, test_len = 480, 40
# train_dataset = PSDataset(list(sam_annotations.keys())[:train_len], device)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True) 
# test_dataset = PSDataset(list(sam_annotations.keys())[train_len:train_len+test_len], device)
# val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True) 

### training

In [11]:
# import wandb
# from pytorch_lightning.loggers import WandbLogger
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()

# epochs = 10
# wandb.login(key='3d1ea92c9996d6ca0b364af566e085cb8dccae9d')
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="Capillaries",
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 1e-3,
#     "architecture": "SAM-finetune mask decoder",
#     "dataset": "Only 20 images",
#     "epochs": epochs,
#     }
# )
# wandb_logger = WandbLogger()

In [12]:
# Consider changing the loss function to Dice Loss or BCE Loss
# class DiceLoss(torch.nn.Module):
#     def __init__(self, eps=1e-5):
#         super().__init__()
#         self.eps = eps
        
#     def forward(self, output, target):
#         intersection = (output * target).sum()
#         union = output.sum() + target.sum() + self.eps
#         dice_score = 2.0 * intersection / union
#         return -(1.0 - dice_score)

# loss_fn = DiceLoss()
loss_fn = torch.nn.BCELoss()

In [13]:
# import pytorch_lightning as pl
# from torch.nn.functional import threshold, normalize
# from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
# from torchvision.utils import make_grid

# class MyModel(pl.LightningModule):
#     def __init__(self, sam, loss_fn, plot_every_x_steps):
#         super(MyModel, self).__init__()
#         self.sam = sam
#         self.loss_fn = loss_fn
#         self.plot_every_x_steps = plot_every_x_steps
#         self.transform = ResizeLongestSide(sam.image_encoder.img_size)
    
#     def forward_pass(self, transformed_data, ground_truth_masks, bbox_coords):
#         input_image = transformed_data['image'].to(self.device)
#         input_size = transformed_data["input_size"]
#         original_image_size = transformed_data["original_image_size"]
#         input_size = (input_size[0].item(), input_size[1].item())
#         original_image_size = (original_image_size[0].item(), original_image_size[1].item())
#         losses = []

#         for i in range(len(ground_truth_masks)):
#             ground_truth_masks[i] = ground_truth_masks[i][0].to(self.device)
#             prompt_box = bbox_coords[i][0].cpu().numpy()
            
#             with torch.no_grad():
#                 image_embedding = self.sam.image_encoder(input_image)
#                 box = self.transform.apply_boxes(prompt_box, original_image_size)
#                 box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)
#                 box_torch = box_torch[None, :]

#                 sparse_embeddings, dense_embeddings = self.sam.prompt_encoder(
#                     points=None,
#                     boxes=box_torch,
#                     masks=None,
#                 )

#             low_res_masks, _ = self.sam.mask_decoder(
#                 image_embeddings=image_embedding,
#                 image_pe=self.sam.prompt_encoder.get_dense_pe(),
#                 sparse_prompt_embeddings=sparse_embeddings,
#                 dense_prompt_embeddings=dense_embeddings,
#                 multimask_output=False,
#             )

#             upscaled_masks = self.sam.postprocess_masks(low_res_masks, input_size, original_image_size).to(self.device)
#             probability_mask = torch.sigmoid(upscaled_masks[0,0])
            
#             loss = self.loss_fn(probability_mask, ground_truth_masks[i].float())
#             losses.append(loss)
#             last_upscaled_mask = upscaled_masks

#         mean_loss = torch.stack(losses).mean()

#         return mean_loss, last_upscaled_mask

#     def training_step(self, batch, batch_idx):
#         transformed_data, ground_truth_masks, bbox_coords = batch
#         if len(ground_truth_masks) == 0:
#             return None
#         self.last_imgs = transformed_data['image'] 
#         self.last_masks = ground_truth_masks[-1]
#         mean_loss, last_upscaled_mask = self.forward_pass(transformed_data, ground_truth_masks, bbox_coords)
#         self.last_upscaled_masks = last_upscaled_mask
#         #self.log('train_loss', mean_loss)
#         self.log("train_loss", mean_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log_images()
#         return mean_loss

#     @torch.no_grad()
#     def validation_step(self, batch, batch_idx):
#         transformed_data, ground_truth_masks, bbox_coords = batch
#         if len(ground_truth_masks) == 0:
#             return None
#         self.last_imgs = transformed_data['image'] 
#         self.last_masks = ground_truth_masks[-1]
#         mean_loss, last_upscaled_mask = self.forward_pass(transformed_data, ground_truth_masks, bbox_coords)
#         self.last_upscaled_masks = last_upscaled_mask
#         self.log('val_loss', mean_loss)
#         self.log_images()
#         return mean_loss   
    
#     def configure_optimizers(self):
#         lr = 1e-3
#         wd = 0
#         optimizer = torch.optim.Adam(self.sam.mask_decoder.parameters(), lr=lr, weight_decay=wd)
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

#         return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "train_loss"}
     
#     def log_images(self):
#         if True or self.global_step % self.plot_every_x_steps == 0:  # plot every x epochs
#             last_img = self.last_imgs[0].cpu().numpy().transpose(1,2,0)
#             last_mask = self.last_masks[0].detach().cpu().numpy()
#             last_upscaled_mask = self.last_upscaled_masks[0,0].detach().cpu().numpy()
#             self.logger.experiment.log({
#                 "Input Images": wandb.Image(last_img),
#                 "Ground Truth Masks": wandb.Image(last_mask, mode='L'),
#                 "Predicted (upscaled) Masks": wandb.Image(last_upscaled_mask, mode='L'),
#             })
        

# model = MyModel(sam, loss_fn, plot_every_x_steps=10)
# checkpoint_callback = ModelCheckpoint(
#     dirpath="checkpoints",
#     filename="best-checkpoint",
#     save_top_k=1,
#     verbose=True,
#     monitor="val_loss",  # assumes you have a validation step where you log a "val_loss"
#     mode="min",
#     every_n_epochs=5,  # change this to save every X epochs
# )
# early_stopping_callback = EarlyStopping(
#     monitor="val_loss",  # assumes you have a validation step where you log a "val_loss"
#     patience=3,
#     mode='min'
# )
# trainer = pl.Trainer(
#     logger=wandb_logger,
#     max_epochs=epochs,
#     callbacks=[checkpoint_callback, early_stopping_callback],
#     log_every_n_steps=10,
# )
# trainer.fit(model, train_loader, val_loader)
# wandb.finish()

### generated frames

In [14]:
mkdir /kaggle/working/frames

In [15]:
STRIDE = 0.1
MAX_IMAGE_SIZE = 1024

def get_frames_from_video(video_file, stride=1.0):
    """
    video_file - path to file
    stride - i.e 1.0 - extract frame every second, 0.5 - extract every 0.5 seconds
    return: list of images, list of frame times in seconds
    """
    video = cv2.VideoCapture(video_file)
    fps = video.get(cv2.CAP_PROP_FPS)
    i = 0.
    images = []
    frame_times = []

    while video.isOpened():
        ret, frame = video.read()
        if ret:
            images.append(frame)
            frame_times.append(i)
            i += stride
            video.set(1, round(i * fps))
        else:
            video.release()
            break
    return images, frame_times


def resize_if_necessary(image, max_size=1024):
    """
    if any spatial shape of image is greater 
    than max_size, resize image such that max. spatial shape = max_size,
    otherwise return original image
    """
    if max_size is None:
        return image
    height, width = image.shape[:2]
    if max([height, width]) > max_size:
        ratio = float(max_size / max([height, width]))
        image = cv2.resize(image, (0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_CUBIC)
    return image


def demo_generate_frames():
    sample_video = '../input/player-segmentation/Test.mp4'
    images, frame_times = get_frames_from_video(sample_video, STRIDE)
    images = [resize_if_necessary(image, MAX_IMAGE_SIZE) for image in images]

    return images


def generate_frames(input_path, save_path, stride, max_image_size):

    images, frame_times = get_frames_from_video(input_path, stride)
    images = [resize_if_necessary(image, max_image_size) for image in images]
    for image, frame_time in zip(images, frame_times):
        image_name = str(round(frame_time, 3)).replace(".", "_")
        ssave_path = os.path.join(save_path, "{}.jpg".format(image_name))
        print('saving:  ', ssave_path)
        cv2.imwrite(ssave_path, image)  


sample_video = '../input/player-segmentation/Test.mp4'
frame_save_path = "/kaggle/working/frames"

generate_frames(sample_video, frame_save_path, STRIDE, MAX_IMAGE_SIZE)

saving:   /kaggle/working/frames/0_0.jpg
saving:   /kaggle/working/frames/0_1.jpg
saving:   /kaggle/working/frames/0_2.jpg
saving:   /kaggle/working/frames/0_3.jpg
saving:   /kaggle/working/frames/0_4.jpg
saving:   /kaggle/working/frames/0_5.jpg
saving:   /kaggle/working/frames/0_6.jpg
saving:   /kaggle/working/frames/0_7.jpg
saving:   /kaggle/working/frames/0_8.jpg
saving:   /kaggle/working/frames/0_9.jpg
saving:   /kaggle/working/frames/1_0.jpg
saving:   /kaggle/working/frames/1_1.jpg
saving:   /kaggle/working/frames/1_2.jpg
saving:   /kaggle/working/frames/1_3.jpg
saving:   /kaggle/working/frames/1_4.jpg
saving:   /kaggle/working/frames/1_5.jpg
saving:   /kaggle/working/frames/1_6.jpg
saving:   /kaggle/working/frames/1_7.jpg
saving:   /kaggle/working/frames/1_8.jpg
saving:   /kaggle/working/frames/1_9.jpg
saving:   /kaggle/working/frames/2_0.jpg
saving:   /kaggle/working/frames/2_1.jpg
saving:   /kaggle/working/frames/2_2.jpg
saving:   /kaggle/working/frames/2_3.jpg
saving:   /kaggl

In [16]:
import ipywidgets as widgets


def show_anns(anns, axes=None):
    if len(anns) == 0:
        return
    if axes:
        ax = axes
    else:
        ax = plt.gca()
        ax.set_autoscale_on(False)
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.5)))
#---------------------------------------------------       
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='red', marker='o', s=80, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='o', s=80, edgecolor='white', linewidth=1.25)  
#---------------------------------------------------
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=1)
    else:
        color = np.array([200/255, 0/255, 0/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
#---------------------------------------------------   
def custom_plot(title,image,specific_point):
    input_label = np.array([1])
    #--------------
    masks = mask_generator_2.generate(image)
    
    print('masks')
    #--------------
    predictor.set_image(image)
    masks_p, scores, logits = predictor.predict(
        point_coords=specific_point,
        point_labels=input_label,
        multimask_output=True,
    )
    return masks, masks_p, scores, logits

def draw(img_id):    
    polygon = polygon_map[img_id]
    img = cv2.imread(img_map[img_id])

    blood_vessel = 0
    glomerulus = 0
    unsure = 0
    annotations = []
    for anno in polygon['annotations']:
        if anno['type'] == 'blood_vessel':
            color = (0,255,0)
            blood_vessel += 1
            
        elif anno['type'] == 'glomerulus':
            color = (0,0,0)
            glomerulus += 1
        else:
            color = (255,0,0)
            unsure += 1

        pts = anno['coordinates']
        pts = np.array(pts)
        pts = pts.reshape(-1, 1, 2)
        annotations.append(pts)
        cv2.polylines(img, pts, True, color, 3)
    
    print(f'{blood_vessel = }')
    print(f'{glomerulus = }')
    print(f'{unsure = }')

    plt.imshow(img)
    return annotations

#----------
def draw_sam(img_name):
    image = cv2.cvtColor(
        cv2.imread('/kaggle/working/frames/' + img_name), 
        cv2.COLOR_BGR2RGB
    )
    specific_point = np.array([[50, 100]])
    (aa, a, b, c) = custom_plot(img_name, image, specific_point)
    return (aa, a, b, c)
    
output = widgets.Output()


In [17]:
mkdir /kaggle/working/frames_out2

In [18]:
test_image_folder = '/kaggle/working/frames' 
predict_save_path = "/kaggle/working/frames_out2"

test_images = [img for img in os.listdir(test_image_folder)
              if img.endswith(".jpg") or
                 img.endswith(".jpeg") or
                 img.endswith("png")]

for img in tqdm(test_images):
    print('img', img)
    (aa, a, b, c) = draw_sam(img)
    
    fin = None
    for i in aa:
        if fin is None :
            fin = i['segmentation']
        else:
            fin += i['segmentation']

    spredict_save_path = os.path.join(predict_save_path, "{}_out.jpg".format(img.split('.')[0]))
    print('saving:  ', spredict_save_path)
    aa = 1.0 * fin
    aa = (aa*255).astype(np.uint8)
    cv2.imwrite(spredict_save_path, aa)

  0%|          | 0/340 [00:00<?, ?it/s]

img 14_2.jpg
masks


  0%|          | 1/340 [00:20<1:53:20, 20.06s/it]

saving:   /kaggle/working/frames_out2/14_2_out.jpg
img 8_9.jpg
masks


  1%|          | 2/340 [00:36<1:40:47, 17.89s/it]

...

saving:   /kaggle/working/frames_out2/29_9_out.jpg
img 26_8.jpg
masks


100%|█████████▉| 339/340 [1:33:32<00:17, 17.45s/it]

saving:   /kaggle/working/frames_out2/26_8_out.jpg
img 33_9.jpg
masks


100%|██████████| 340/340 [1:33:51<00:00, 16.56s/it]

saving:   /kaggle/working/frames_out2/33_9_out.jpg
